In [1]:
import numpy as np 
import pandas as pd

In [2]:
df = pd.read_csv("songdata.csv")
df.head()

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \nAnd..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \nTouch me gentl..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \nWhy I had t...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


In [3]:
df.shape

(57650, 4)

In [4]:
df = df.sample(n=5000).drop('link',axis=1).reset_index(drop=True)

#recommendation based on popularity, content base , collaborative filtering,hybrid 

#Clean

In [5]:
df['text'][0]

"Oh so, my love  \nThis sleeping pill  \nWill watch you through the night  \nAnd come dawn  \nI'll be long, long gone  \nI tried, I tried, I tried  \nI.. Tried, i.. Tried  \nWe feed upon the broken branch  \nWe kiss the stars goodbye  \nFor the sun can't fight this overcast  \nI tried, I tried, I tried  \nI.. Tried, i.. Tried  \nThe spider's web  \nIt can't break our fall  \nI whisper to the sky  \nI pray she takes you in her arms  \nI tried, I tried, I tried  \nI.. Tried, i.. Tried  \nI.. Tried..  \nStorm warning in the sand  \nShe evaporates beneath my fingernails  \nThe road is lost underwater  \nShe's running down a river with eels  \nShe is lost to me.\n\n"

In [6]:
df['text'].str.lower().replace(r'[^\w\s]',"").replace(r'\n','',regex=True)

0       oh so, my love  this sleeping pill  will watch...
1       praise him you heavens and all that's above  p...
2       sittin' here on terra firma  it's the kind of ...
3       somedays, we forget to look around us,  someda...
4       when i took a little loving from you  oh i, ne...
                              ...                        
4995    oh, oh, the sun beats down  and burns the tar ...
4996    everybody's gone away  said they're movin' to ...
4997    june was the lobster shell,  i dug by hand, a ...
4998    (rock mafia)  drop the music,  coast to coast....
4999    i'm hungry for your love  i'm hungry for your ...
Name: text, Length: 5000, dtype: object

In [7]:
import nltk
# nltk.download('punkt')
from nltk.stem.porter import PorterStemmer

ps = PorterStemmer()

def tokenization(txt):
    tokens = nltk.word_tokenize(txt)
    stemming = [ps.stem(w) for w in tokens] 
    return " ".join(stemming)


In [8]:
tokenization("loving love loved")

'love love love'

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity



In [10]:
tfid = TfidfVectorizer(stop_words='english')
matrix = tfid.fit_transform(df['text'])

In [11]:
matrix.shape

(5000, 24001)

In [12]:
similarity = cosine_similarity(matrix)

In [13]:
similarity[0]

array([1.        , 0.00740086, 0.00701904, ..., 0.02168997, 0.02205137,
       0.01468748])

In [14]:
df['song'][0]

'Sleeping Pill'

In [15]:
df['song']=='Cashed Out'

0       False
1       False
2       False
3       False
4       False
        ...  
4995    False
4996    False
4997    False
4998    False
4999    False
Name: song, Length: 5000, dtype: bool

In [16]:
df[df['song']=='Cashed Out']

,artist,song,text
4269,Lil Wayne,Cashed Out,Fuck nigga \nMy blunt bigger than these lil n...


In [17]:
list(enumerate(similarity[0]),reverse=False,key=lambda x:x[1])

TypeError: list() takes no keyword arguments

In [18]:
def recommendation(song):
    idx = df[df['song']==song].index[0]
    distances = sorted(list(enumerate(similarity[idx])),reverse=False,key=lambda x:x[1])
    songs = []
    for i in distances[1:21]:
        songs.append(df.iloc[i[0]].song)
    return songs
    

In [20]:
recommendation('Cashed Out')

['No Respect',
 'Because We Can',
 'Shining',
 'Endless',
 'Country Darkness',
 'Darkness Of Christ',
 'God Is Moving',
 'One Of These Things First',
 'Yachil Vi Veyachali',
 'Oh Christmas Tree',
 'Hagkan',
 'Along The Road',
 "Es Ist Ein Ros' Entsprungen",
 'Starfuckers',
 'Abide With Me',
 'West Of The Moon',
 'Greater Than All',
 'Aku Disini',
 'Moonlight In Vermont',
 'Always']

In [21]:
import pickle
pickle.dump(df,open('df.pkl','wb'))
pickle.dump(similarity,open('similarity.pkl','wb'))